In [1]:
import torch
import torch.nn as nn
import torchvision
import torch.utils.data
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F

from PIL import Image
from torchvision import transforms
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import accuracy_score, roc_auc_score

%matplotlib inline

In [2]:
device = torch.device('cuda')

In [3]:
transform = transforms.Compose([
    transforms.Resize((227, 227)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
train_data = torchvision.datasets.ImageFolder(r'D:\PycharmProjects\NNs\CNN\datasets\train_folder', transform=transform)
test_data = torchvision.datasets.ImageFolder(r'D:\PycharmProjects\NNs\CNN\datasets\test_folder', transform=transform)

In [4]:
batch_size = 32
train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_data_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=True)

In [5]:
class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet, self).__init__()
        self.l = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=96, kernel_size=11, stride=4),
            nn.BatchNorm2d(96),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(in_channels=96, out_channels=256, kernel_size=5, padding=2),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(in_channels=256, out_channels=384, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(384),
            nn.ReLU(),
            nn.Conv2d(in_channels=384, out_channels=384, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(384),
            nn.ReLU(),
            nn.Conv2d(in_channels=384, out_channels=256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2)
        )

        self.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(9216, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Linear(4096, 2)
        )

    def forward(self, x):
        x = self.l(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

In [6]:
model = AlexNet().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, weight_decay=0.005, momentum=0.9)

In [7]:
def fit(train_dataset: torch.Tensor, n_epochs=20):
    for epoch in range(n_epochs):
        pred_per_epoch = []
        true = []
        for i, batch in enumerate(train_dataset):
            inputs, target = batch

            inputs = inputs.to(device)
            target = target.to(device)

            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            true += target.tolist()
            pred_per_epoch += predicted.tolist()
            loss = criterion(outputs, target)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch + 1, n_epochs, i + 1, len(train_dataset), loss.item()))
        print("Accuracy at {} epoch: {:.2f}%".format(epoch + 1, accuracy_score(true, pred_per_epoch)))

In [8]:
def score(dataset: torch.Tensor):
    y_pred = []
    y_true = []
    for inputs, target in dataset:
        inputs = inputs.to(device)
        target = target.to(device)

        predict = model(inputs)
        _, predicted = torch.max(predict.data, 1)
        y_true += target.tolist()
        y_pred += predicted.tolist()

    print("Accuracy: {}%. ROC-AUC: {:.2f}".format(accuracy_score(y_true, y_pred), roc_auc_score(y_true, y_pred)))

In [9]:
def predict(path):
    target = ['smile', 'not smile']
    img = Image.open(path)
    img = transform(img).to(device)
    img = torch.unsqueeze(img, 0)

    model.eval()
    prediction = torch.max(model(img), 1).indices.item()
    print(target[prediction])

In [10]:
fit(train_data_loader)

Epoch [1/20], Step [1/88], Loss: 0.6578
Epoch [1/20], Step [2/88], Loss: 0.7792
Epoch [1/20], Step [3/88], Loss: 0.6998
Epoch [1/20], Step [4/88], Loss: 0.6740
Epoch [1/20], Step [5/88], Loss: 0.6642
Epoch [1/20], Step [6/88], Loss: 0.6990
Epoch [1/20], Step [7/88], Loss: 0.6978
Epoch [1/20], Step [8/88], Loss: 0.6774
Epoch [1/20], Step [9/88], Loss: 0.6774
Epoch [1/20], Step [10/88], Loss: 0.7389
Epoch [1/20], Step [11/88], Loss: 0.6685
Epoch [1/20], Step [12/88], Loss: 0.6763
Epoch [1/20], Step [13/88], Loss: 0.6731
Epoch [1/20], Step [14/88], Loss: 0.7107
Epoch [1/20], Step [15/88], Loss: 0.7001
Epoch [1/20], Step [16/88], Loss: 0.6470
Epoch [1/20], Step [17/88], Loss: 0.7621
Epoch [1/20], Step [18/88], Loss: 0.6863
Epoch [1/20], Step [19/88], Loss: 0.6671
Epoch [1/20], Step [20/88], Loss: 0.6738
Epoch [1/20], Step [21/88], Loss: 0.6664
Epoch [1/20], Step [22/88], Loss: 0.6933
Epoch [1/20], Step [23/88], Loss: 0.6442
Epoch [1/20], Step [24/88], Loss: 0.7033
Epoch [1/20], Step [25/88

In [11]:
score(test_data_loader)

Accuracy: 0.9%. ROC-AUC: 0.90


In [12]:
predict("1.jpg")

smile


In [13]:
torch.save(model, 'AlexNet')